In [129]:
from nltk.tokenize import sent_tokenize, word_tokenize
import random
import nltk
from operator import itemgetter
import re
import numpy as np

In [130]:
def addtG(TOKEN):
    
    for i in range (len(TOKEN)):
        TOKEN[i] = re.sub(r'^',"<s>",TOKEN[i])
        TOKEN[i] = re.sub(r'$',"</s>",TOKEN[i])
    #print(TOKEN)


In [131]:
#unigram MLE
def uniP(freq):
    numTok = len(unigrams)
    prob = {}
    for (key,fvalue) in reversed(sorted(freq.items(), key = itemgetter(1))):
        prob[key] = fvalue/numTok
    
    return prob


In [132]:
def textProc(file_name):
    f= open(file_name,'r')
    #tp1
    corpus = f.read().replace('\n',' ') #replace the new line with single white space.
    corpus = corpus.lower() #tp2

    token = sent_tokenize(corpus)

    #replace all the non alphanumeric characters with whitespace.
    token = [re.sub(r'[^A-Za-z0-9]'," ", i) for i in token]#tp3

    random.shuffle(token)

    #addtG(token) #add tags
    return token

    #clean tokens of sentence returned


In [133]:
#write a function for count of n-gram
def LstOfNgram(count,corpus):
    #cleaned corpus
    n_gram=[]
    for i in corpus:
        p = word_tokenize(i)
        p = ["<s>"]+p
        p.append("</s>")    
        #token must have start and end line tags here.
        #print (p)
        for g in range(count-1,len(p)):
            dummy=""
            for j in range(count):
                dummy = p[g-j]+" "+dummy   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
               #more things to be added here
            dummy = re.sub(r"^\s|\s$","",dummy)
            n_gram.append(dummy)
    return (n_gram)    

    

In [134]:
def CountNgram(frequency):
    count ={}
    
    for (key,fvalue) in reversed(sorted(frequency.items(), key = itemgetter(1))):
        count[key] = fvalue
 
    return count

In [135]:
#unified method for MLE of n-gram
def MLE(a,b,N):  #n is N of n-gram
    if N ==1:#unigram
        return (count_unigram[a]/len(unigramLst))
    elif N==2:#bigram
        return (count_bigram[b+" "+a]/count_unigram[b])
    elif N == 3:#triram
        return (count_trigram[b+" "+a]/count_bigram[b])
    elif N == 4:
        return (count_quadgram[b+" "+a]/count_trigram[b])

In [136]:
#question 5
def add_1_prob(a,b):#specificaly for bigrams
    try:
        numerator = count_bigram[b+" "+a] + 1
    except :
        numerator = 1
    try:   
        denominator = count_unigram[b] + len(count_unigram) #count_unigram is count of unigram(types in corpus)
    except :
        denominator = len(count_unigram)
    return (numerator/denominator)
#assure once that it is len(count_bigam)

In [137]:
def get_NLst(count_dict):#pass dicationary of word vs count.
    #create counting list for 
    l = count_dict 
    l=list(l.values())
    l.sort()
    #we want a dictionary with {1:1,2:1,3:2,4:3,5:1}
    d={}
    for i in range(len(l)):
        count =1
        for j in range(i,0,-1):
            if l[i] == l[j-1]:
                count = count + 1
            else:
                break 
    
        d[l[i]] = count
    return (d)
#takes O(n^2) which takes lot of time to execute.
    

In [164]:
class goodTurning:
    def __init__(self,count): 
        self.countNgram = count
        self.dictN = get_NLst(self.countNgram)
        self.keys = list(dictN.keys())
        self.values = list(dictN.values())
        self.NewCount()
    def NewCount(self):
        self.newCount=self.countNgram.copy()
        
        for i in self.newCount.keys():
            try:
                self.newCount[i] = (self.newCount[i]+1)*(self.dictN[self.newCount[i]+1]/self.dictN[self.newCount[i]])
            except:
                intr= np.interp(float(self.newCount[i]),[float(j) for j in self.keys],[float(k) for k in self.values])
                #intr= np.interp(l[i]+1,keys,values)

                self.newCount[i] = (self.newCount[i]+1)*(intr/(self.dictN[self.newCount[i]]))
            #new for the word with zero count 

        return self.newCount 
    def smoothProb(self,a,token): 
        #token : total number of token of n-gram 
    
        if  a not in self.countNgram.keys():
            return (dictN[1]/len(token)) #count of 1 divided by total ngrams in the corpus
        
        else:
            return (self.NewCount()[a]/len(token))
#look over interpolation function
#for Nc >Nc+1 it is too jumpy
        

In [139]:

token = textProc('SherlockHomes.txt')


train = token[:int(len(token)*0.8)]#80%
validation = token[int(len(token)*0.8):]#20%
#print (token)


In [140]:
unigrams = []
for i in train:#token is token of sentence.
    p=word_tokenize(i)
    p = ["<s>"]+p
    p.append("</s>")
    unigrams+=p
#HOW TO GET RIDGE OF THIS 
#print (unigrams)
#MLE of unigram
#frequency of occurence divided by total number of tokens
dict_freq = nltk.FreqDist(unigrams)
MLE_unigram = uniP(dict_freq)

#print (MLE_unigram)


In [141]:
#n-gram count
#token = ["this is a sentence","this is something differnt","this is something differnt that this has happend"]
#list contining n-gram
unigramLst = LstOfNgram(1,train)  ##the first number here representes the N of N-gram.
bigramLst  = LstOfNgram(2,train)
trigramLst = LstOfNgram(3,train)
quadgramLst= LstOfNgram(4,train)
#print (bigramLst)
#  query for count of unigram only once
feq  =  nltk.FreqDist(unigramLst)
freq =  nltk.FreqDist(bigramLst)
fre  =  nltk.FreqDist(trigramLst)
fr   =  nltk.FreqDist(quadgramLst)

#count_unigram={}
count_unigram = CountNgram(feq)
count_bigram = CountNgram(freq)
count_trigram = CountNgram(fre)
count_quadgram = CountNgram(fr)
print (count_unigram)


{'</s>': 5443, '<s>': 5443, 'the': 4472, 'i': 2446, 'and': 2396, 'to': 2238, 'a': 2115, 'of': 2075, 'in': 1392, 'that': 1380, 'it': 1359, 'he': 1179, 'you': 1167, 'was': 1151, 'his': 913, 'is': 876, 'my': 800, 'have': 747, 'as': 684, 'with': 660, 'had': 638, 'at': 619, 'which': 610, 'for': 588, 'not': 535, 'but': 518, 'me': 510, 'be': 497, 'we': 443, 'this': 417, 'said': 396, 'there': 391, 'from': 372, 'holmes': 366, 'upon': 363, 'him': 360, 'so': 356, 'her': 329, 'she': 327, 'all': 322, 'very': 322, 'been': 314, 'on': 311, 'your': 309, 'no': 305, 'what': 302, 'one': 299, 'then': 294, 's': 294, 'were': 285, 'by': 277, 'are': 267, 'would': 264, 'when': 263, 'out': 260, 'an': 260, 'man': 253, 'up': 246, 'do': 235, 'mr': 222, 'has': 219, 'little': 219, 'could': 217, 'who': 215, 'into': 214, 'will': 210, 'if': 206, 'some': 197, 'now': 187, 'see': 182, 'down': 178, 'should': 174, 'our': 172, 'can': 171, 'they': 165, 'well': 164, 'may': 161, 'or': 156, 'us': 146, 'over': 143, 'think': 140, '

In [165]:
#n-gram evalutation, how many n-grams are there
#question 6 
#p =good_turing(count_bigram)
p = goodTurning(count_bigram)
p.NewCount()
#goodTuning smooting trained

0.7537390717949953


In [ ]:
#question 3 and question 
f1 = MLE("a","",1)
f2 = MLE("a","is",2)#probability of bigram 
f3 = MLE("a", "this is",3)
#print(f1)
#print (f2)
#print (f3)
print (MLE("is","this",2))

#MLE_bigram("this","is") "this" given "is" = count("is this")/count(is)

#finding quadgram of a small courpus is of no use do appears to be a good idea.
#p = good_turing(count_bigram)
#p


#how to do the second part?

In [ ]:
#QUESTION 5
print (add_1_prob("is","this"))#discounted probability
print (add_1_prob("nott","here in"))#new probabilited to the one that is not present
print (add_1_prob("not","here "))#new probabilited to the one that is not present

In [ ]:
#question 7
def prePlexity()

In [ ]:
#rough work below[a]/count_unigram

In [ ]:
l="<s>this is a sentence here is.</s>"
token = word_tokenize(l)
#print (token)
regx = re.compile("[a-z0-9]+ [a-z0-9]+")
r =re.findall(regx,l)
p= ["<s>this", "this is","is a ","a sentence","sentence here","here is","is. </s>"]
print (r)

In [ ]:



import re
l = "this is a sentence here is"
token1 = word_tokenize(l)
token1 = ["<s>"]+token1
token1.append("</s>")
n_gram = []
n=2 #bigram 
#so keeping a sentence token is a good idea
#for i in range(n-1,len(token)):#start with the n-1 of the n gram, so that it does not give any error
 #   n_gram.append(token[i-1]+" "+token[i])
#print (n_gram)

#for n gram              <s>this is a sentence here is</s>
print (token1)
for i in range(n-1,len(token1)):
    p=" "
    for j in range(n):
       p = token1[i-j]+" "+p   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
       #more things to be added here
    n_gram.append(p)
#print (n_gram)

In [ ]:
1

In [ ]:
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}



In [ ]:
l=[1,2,3,3,4,4,4,5,5,5,5,5,5,7,8,8,10,10,10,11,12]
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}

l=list(l.values())
l.sort()

#we want a dictionary with {1:1,2:1,3:2,4:3,5:1}
d={}
for i in range(len(l)):
    count =1
    for j in range(i,0,-1):
        if l[i] == l[j-1]:
            count = count + 1
          
        else:
            break 
    
    d[l[i]] = count
d

In [ ]:
p = get_NLst()
p


In [ ]:
print (p)

In [ ]:
l=[1,2,3,4,5]
p=l.copy()
p[0]= 2
p



In [ ]:
import numpy as np
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}

dictN = get_NLst(l)#dictionary of all the N's and corresponding values

keys = list(dictN.keys())
values = list(dictN.values())
for i in l.keys():
    try:
       
        l[i] = (l[i]+1)*(dictN[l[i]+1]/dictN[l[i]])
    except:
        intr = np.interp(l[i]+1,keys,values)
        
        l[i] = (l[i]+1)*(intr/dictN[l[i]])
l


In [ ]:
l = ["<s>this is a sentence for me<\s>","<s>that is a sentence for you man<\s>"]
#inverse of all the bigram probability to the power N,
#probability of all the MLE birams
#define a function that takes the sentences from the test data set and give out ptobability.

